In [1]:
import serial
import time

class AM4100:
    def __init__(self, com_port='COM4', baud_rate=115200):
        self.com_port = com_port
        self.baud_rate = baud_rate
        self.serial_port = None
        self.setup_serial_connection()

    def setup_serial_connection(self):
        try:
            self.serial_port = serial.Serial(self.com_port, self.baud_rate, timeout=10)
            self.serial_port.bytesize = serial.EIGHTBITS
            self.serial_port.stopbits = serial.STOPBITS_ONE
            self.serial_port.parity = serial.PARITY_ODD
            self.serial_port.write_timeout = 10  # Set write timeout to prevent blocking write calls
            time.sleep(1)  # Allow some time for the serial port to initialize
        except serial.SerialException as e:
            print(f"Error opening serial port: {e}")
            self.serial_port = None

    def flush_serial_port(self):
        if self.serial_port is not None:
            self.serial_port.reset_input_buffer()  # Clear input buffer
            self.serial_port.reset_output_buffer()  # Clear output buffer

    # def send_command_to_stimulator(self, command):
    #     if self.serial_port is not None:
    #         try:
    #             self.serial_port.write(command.encode() + b'\r\n')
    #             print(f"Send= {command}")
    #             #time.sleep(0.1)  # Short delay to allow command processing
    #             response = self.serial_port.readline().decode().strip()
    #             response = response.replace('\r', '').replace('\n', '~')  # Process response
    #             print(f"Reply= {response}")
    #             return response
    #         except serial.SerialException as e:
    #             print(f"Error sending command: {e}")
    #             return "error no data"
    #     else:
    #         print("Serial port is not open.")
    #         return "error no data"
    def send_command_to_stimulator(self, command, terminator=b'\n'):
        if self.serial_port is not None:
            try:
                self.serial_port.write(command.encode() + b'\r\n')  # Send the command with CR+LF
                print(f"Send= {command}")
                response = self.serial_port.read_until().decode().strip()  # Read until the terminator
                
                response = response.replace('\r', '').replace('\n', '~')  # Process response to replace CR and LF with tilde
                print(f"Reply= {response}")
                return response
            except serial.SerialException as e:
                print(f"Error sending command: {e}")
                return "error no data"
        else:
            print("Serial port is not open.")
            return "error no data"
    
    def send_command_and_read_response(self, command):
        if self.serial_port is not None:
            try:
                # Send command
                full_command = command.encode() + b'\r\n'
                self.serial_port.write(full_command)
                print(f"Send= {command}")

                # Read response until the expected terminator
                response = self.serial_port.read_until(b'*\r\n')
                decoded_response = response.decode().strip()

                # Print the raw response for debugging
                print(f"Raw Reply= {decoded_response}")

                # Process and extract the relevant part of the response
                # The value after the last newline and before the '*'
                relevant_part = decoded_response.split('\r\n')[-2]  # Get the second last item after split which should be `-25`
                print(f"Relevant Response= {relevant_part}")
                return relevant_part
            except serial.SerialException as e:
                print(f"Error sending command or reading response: {e}")
                return "error no data"
        else:
            print("Serial port is not open.")
            return "error no data"
    
    def set_amplitude(self, amplitude):
        command = f"1001 s m 10 7 {amplitude}"
        response = self.send_command_to_stimulator(command)
        return response
    def get_amplitude(self):
        self.serial_port.write('g m 10 7'.encode())
        response = self.serial_port.readline().decode().strip()
        print(f'reply= {response}')
    def read(self):
        response = self.serial_port.read_until()
    def run(self): # this should set it so its waiting for trigger
        command = '1001 s a run'
        response = self.send_command_to_stimulator(command)
    def stop(self): # put it in a ready state but not able to receive trigger
        command = '1001 s a stop'
        response = self.send_command_to_stimulator(command)
        return response

    def close_connection(self):
        if self.serial_port is not open:
            self.serial_port.close()
            print("Serial port closed.")
        else:
            print("Serial port was not open.")

In [3]:
am = AM4100('COM1')
am.flush_serial_port()


In [20]:
am.set_amplitude(-50)


Send= 1001 s m 10 7 -50
Reply= 


''

In [33]:
result = am.send_command_and_read_response('g m 0 0')

Send= g m 0 0
Raw Reply= g m 0 0
0
*
Relevant Response= 0


In [4]:
result = am.send_command_and_read_response('g m 10 7')
print("Extracted Value:", result)

Send= g m 10 7
Raw Reply= g m 10 7
-25
*
Relevant Response= -25
Extracted Value: -25


In [35]:
am.serial_port.write('s m 0 0'.encode() + b'\r\n')
am.serial_port.read_all()

b'\ng m 0 0\r\r\n0\r\n*\r\n'

In [13]:
response = am.send_command_to_stimulator('get status', terminator=b'\r')
print(response)

Send= get status
Reply= get status
get status


In [4]:
am.set_amplitude(amplitude = -25)

Send= 1001 s m 10 7 -25
Reply= 1001 s m 10 7 -25


'1001 s m 10 7 -25'

In [10]:
am.run()

Send= 1001 s a run
Reply= 1001 s a run


In [3]:
am.stop()

Send= 1001 s a stop
Reply= 


''

In [7]:
am.get_amplitude()

reply= 


In [17]:
am.send_command_to_stimulator('1001 s m 0 0 0')

Send= 1001 s m 0 0 0
Reply= *


'*'

In [18]:
am.send_command_to_stimulator('1001 s a stop')

Send= 1001 s a stop
Reply= 


''

In [19]:
am.send_command_to_stimulator('1001 s a stop')

Send= 1001 s a stop
Reply= 1001 s a run


'1001 s a run'

In [35]:
am.send_command_to_stimulator('1001 g m 10 7')

Send= 1001 g m 10 7
Reply= 1001 s a stop


'1001 s a stop'

In [27]:
am.run()
time.sleep(1)
am.send_command_to_stimulator('1001 set trigger one')

Send= 1001 s a run
Reply= 1001 s m 0 0 0
Send= 1001 set trigger one
Reply= *


'*'